<img src="./images/cmiyc2024_test.webp" alt="A kitten with a tophat at a computer. Aka JtR + H" width="800"/>

# Cracking CMIYC 2024 Test Hashes

**GETTING READY FOR CMIYC 2024!!!!**

### Overview

The goal of this notebook is to make it easy as possible for you to get a team registered and ready to participate in CMIYC. I'm hopefull I'll be updating it a lot running up to the contest, so I'd recommend using this as an example, but creating your own notebook to manage your teams cracking sessions. That way you don't have to worry about merge conflicts or overwriting your own work.

### Major Features and Training Goals

1. How to register a team with CMIYC
2. How to decrypt challenge files
3. Still working on this section



### Creating a PGP Key (If Needed)

Let's face it. PGP is a pain, and you don't want to have to try and figure it out during the contest. But all communication with the organizers (KoreLogic) needs to be done via PGP encrypted messages. This includes creating a team and submitting hashes. To make this easier, I created some functions to help using the Python3 **pgpy** library.

The first function is **generate_read_pgp_private_key**. This will create a new PGP key (if one does not already exit) and write it to the key_file name that you send to it. If a PGP key already exists it'll validate the key and then return it. This way you aren't creating a new key every time this is run. Feel free to skip using this function if you already have a PGP key you want to use, but even the conference organizers recommend using a contest specific PGP key to make it easier to partner with other players.

In [1]:
from lib_framework.pgp_mgr import generate_read_pgp_private_key

import os
import sys

# Setting the path this way to make it OS independent
key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'pgp_key.asc'
    )

my_private_key = generate_read_pgp_private_key(key_file, email_address="YOUR_EMAIL_ADDRESS", team_name="YOUR_TEAM_NAME")

### Creating a Registration Email

To make it easier to encrypt/decrypt messages I created the **PGP_Mgr** class. I figured a class would be better than having individual functions since I didn't want to have to keep passing in my private key as well as KoreLogic's public key.

While I could try to integrate this with individual e-mail services, that seems like a lot of work so PGP_Mgr will output encrypted messages to this workbook as well as save the messages to a file. It's up to you then to actually copy/paste (or attach) that message into an e-mail and send it to KoreLogic at sub-2024@contest.korelogic.com

Note, the register_team function will both print the message to your screen as well as return the message. Given the way JupyterLab displays newlines in returned messages I'd recommend hiding the return value using the **";"** at the end of the function call, and copy/pasting the printed version.

In [2]:
from lib_framework.pgp_mgr import PGPMgr, read_pgp_key

# Load up KoreLogic's submisison public key
korelogic_sub_public_key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'defcon-contest-sub-2024.pub.asc'
    )
korelogic_sub_public_key = read_pgp_key(korelogic_sub_public_key_file)

default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'submission.asc'
    )

# Create the PGPMgr class
pgp_mgr = PGPMgr(my_private_key, korelogic_sub_public_key, default_attachment)

# Create Submission e-mail
registration_submission = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'registration_submission.asc'
    )
pgp_mgr.register_team("YOUR_TEAM_NAME", attachment_filename=registration_submission);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf/S08jaSMsH7I4KQj3Uliepi+wa2EQB6HkxoXKamwYIg3b
IrWpbzT7J5RS2pKJ7jNdvVRqjhzcvRfQBZe9w1XQNBULfzUusLtzoENl5wNun1Ou
XnD2irMCubis2EWBa19LJsnTQJkMs+VMYl0dcQK6L4m/GiPhxNm3MipYbZSDzRCm
YNwPbaLazWMThz87YSmMVVuAra7gB4jvKJoFOZavDqs/3djzIe1uqOzuCq9m1sec
hW/xkZWuRpctXOHa7B53M7EU+ute81SKudNzk46GVrxQjvKbJNDvhP82NHP7/O2A
WzrQhq6Flx6iBmNAorS2FKIKykDMS9MDwLYq/w7e59LHVQGYqI2KYHRACx8JTi0P
6BVv3gllo4sF/wNe3ug5rFU99BtU4CdxW50vU/b/rY1c5zpUVf9/+QbwLHs54uRX
VlSEZwv4r9OBIVWsv5FTODVjwldZECYkCQrQK0Y2nQJWrZJclB2725NRbtiZ3ytC
hhvhr0Cnu0VR3eteRUWBZTLo1MV7sAGH6NVNwaRpxnRoAvlXN96Z5n5s194PA/Db
zTi5vxCoSc11XnM206Ywnd1k9fWHorGF5xye4gKZqnzXBkPbQY7cPq2yC1wTI1Da
gBPvs9mvuKAINrQfHXmIZpbsNbRy19CD+lnsz4fb+H+Yu0yTckAjca7ao8Bg12DK
8bEz95BitFalF8NmYbj+3eITYHZO0o9DDPCDShRViJ1EriVEWALY4QvUB/oonCAJ
Dlt2Kkt/ess1eA/piVGfCA+8t6VHjvhFUQ46uWpXQu33Z/4oJf+aIiz6cKpo5wP/
q/0FKmzqkYVSZfqFjfqpfdYFbK8QpsG90j9VNNOYnP7+36PPa+cQ1ctLh1cKNmxx
dK5HdCmMilZisU2DJ7C6v+pGS7l7v8cobfPfv2Ulnfg0U3sycO18oGChYcHqW

### Decrypting the KoreLogic Confirmation Email

If your registration submission was sucessfull, then you should get a follow-up e-mail form KoreLogic with a challenge message in it that you will need to decrypt, sign, and send back to them. To do the decryption you can use the **PGPMgr.decrypt_msg() function.

The tricky part is since this isn't integrated directly into your e-mail client, you'll need to save the PGP encrypted part of the message you recieve to a file for this application to read in.

In [3]:
# Now we need to validate the confirmation e-mail
korelogic_challenge_msg = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms_challenge.asc'
    )
challenge_response = pgp_mgr.decrypt_msg(korelogic_challenge_msg)

Chal: AADD-F342-9A58-0D1C



### Sending Back the Challenge Response to KoreLogic

Once you have the challenge response, you can then encrypt it using the standard **PGPMgr.encrypt_msg() function. This is the function you'll likely be using for most of your conversations with KoreLogic and for submitting your cracked hashes (that is until I automate cracked hash submissions more. This will save the response to a file, which you can then attach to an e-mail and send back to sub-2024@contest.korelogic.com.

Oh and if you are wondering why I'm creating different files for each of these messages, I figured I'd do this in case someone wanted to run this entire notebook so it won't overwrite previous steps. Feel free to use the default filename in the PGPMgr class to only create one file and then run these Jupyter cells one by one for each step.

In [4]:
challenge_response_msg = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms_response.asc'
    )

pgp_mgr.encrypt_msg(challenge_response, attachment_filename=challenge_response_msg);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQgAhYkvjFUlax1TjHGKiWyXAvvJP9ruJ2igVUy04jdJ4Hz6
DlOpOh+K78CBsVZhgf2jXFMYHK6leIgLDM5kbBHVxCXOh5fiPalr9D4AEZMvR0OA
SXYiNmiMza+BdC156etvH/AD96A8vPpgYpydvLJbs7RUQC/HUgMDwFq5YCeT6ndq
FZh811jHlXZ7GeN811pcRBBUl0it8NbuwxMWGEMi4LDfPqQEUC3jW2OFp7lbSJaV
nnXeHE37NZZfvQSm+1M2w2ASG7eG1+HDqt0q9PeQQDStQP6CLcnb1TIDLNup437r
wfWY+tOLxyPjN8McNM8mufiQLuYCpu0oYOtUKkrssNLB2QEGYgmaYDUhVt9PL6eV
hxbnfhsulSrNJeUrQpeI+7UJRRPUAYRfCpsYTd784Y6io01HJNKbO1vn2pCJGhV9
kvhd3j09b0NyV4Wgylu4MvhLvxxvu2OfrEU8D5bhCzLF488K89WsQZ5EAiJxiiPf
9IUTl3JqutyuAMiAiAoMLXGwDhFf9VH2NFeOgPVkDQB9qBmeLtmd2DMNuVZu3zgr
zh8AndXwIUeRg6PEhEke6G8cXeqL0xRB8kwommC8N6Xo+3yXE89+zjsds7fOpj3I
hWxqt+/6r6Z0KV9clhEiSa5pcGKIn/SkZFG/nUhg0FsajlR/eZsb+S7PMY2A+glA
xTfNfGILDFmB7/si3KjRrl/yskHvvaWgSUKGxDS/2xiA9soxcGSlWKvCFf2viRJM
bjhSwVKE2fke5qAINSnFJBSw6zhRjslN9xpIYKzgEsYnZZYz99owMlo/cGfGJPRB
K4JSozdB290YyTT8A5cNtvACgmP8eenAE6AGsFIshxYDeyMqyyAHNmbxKKolzlTz
kPsr2zHhbTKbWEKer0bkl/7q7rFPCBjFcUTjNEluPaaU/8MiJEYeraWSMS9r7

### Decrypting CMIYC Challenge Files

Now that we have a team, let's get to cracking hashes!!!

The first thing to do is to download the challenge files. You can obtain these from the Korelogic website at [https://contest-2024.korelogic.com/downloads.html](https://contest-2024.korelogic.com/downloads.html). I'd recommend saving it in the challenge file directory, but you really can save the challenge files anywhere and just update the file path to point to it.

The good news is the the challenge files are also encrypted with PGP but this time with a passphrase. So we can use much of the same base code. I created a helper function/wrapper to make this a bit easier. The challenge decryption function is **decrypt_challenge_files(filname, passphrase, attachment_filename)**

The attachment_filename is important since if they give you something like an encyrpted zip file you want to save it as a zip file and not have to deal with pasting raw binary into a file...

In [5]:
from lib_framework.pgp_mgr import decrypt_challenge_files


# Decrypting the first challenge file which is just a text file
challenge_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1_passwd.pgp'
    )

saved_challenge1 = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1_passwd.hash'
    )

# Note, unlike a lot of the other functions, this returns a bytearray and not a string
# This is because I have no idea what Korelogic is going to toss at us
test1_hashes = decrypt_challenge_files(
    challenge_file,
    "mXx/Vtt8K$XHv4_a)4ziF*&CnB$ufLyQUJ=iGh<5",
    attachment_filename=saved_challenge1
    )

print(test1_hashes.decode())

user1:e9fa198e766d7b9bdfdb872032b2c6ee
user2:{SSHA}4Sq+ps0Yeo9HvPV45aeF8Z4wWa1vUlpnTkZIRg==
user3:$1$mjlHQY$7ofnyhSdWnn6HyIMf8GIE0
user4:$1$h2lb97w8$rSnzQB3FHv9o4VXaOTEyx.
user5:$2a$08$SBTibTPJWVGsJzXiaVOzZO9cp8DXeaaQtTCdVe8QxT75N.G2nuk76



In [6]:
# Now we will decrypt the second test challenge file which decrypts to a zip file
challenge_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1.zip.pgp'
    )

saved_challenge2 = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1.zip'
    )

# Not saving the results into a variable since it's a zip and I don't have time to include
# a python zip extractor into this framework
decrypt_challenge_files(
    challenge_file,
    "&<J>n+MfiY(~Zfg6$:u{t8[&Vh*>iWN;Oo6N;DJ,",
    attachment_filename=saved_challenge2
    );

### Unzip the Zipped Challenge Files

You should have a test_1.zip file in your hashes directory now. You'll need to manually unzip it and copy the challenge files into the hashes directory. 

**Oh wait! The zip itself is password protected. I guess we need to crack that as well :)**

What you'll really need to do is extract the password hash from the encrypted zip file. Luckily I wrote a blog post on how to do this for multiple different file types, so you can reference it during the contest. Here is the link: [https://reusablesec.blogspot.com](https://reusablesec.blogspot.com/2023/11/jupyter-lab-framework-example.html)

Using the John the Ripper Zip2John extension (and specifying the full file path for the zip file and the output since I don't know where you installed JtR), you can use the following command to extract the hash:

`zip2john test_1.zip >> test1.hash`

### A Brief Side-Tangent on Having Multiple Jupyter Notebooks for Different Tasks

Now here is a point where I'm going to recommend that you do something different than what I'm doing in this Notebook. Registering a team and unpacking contest hashes are really one-time events per contest. Having them in a notebook makes them easy to run and debug which is nice, but as we move on to actually cracking hashes we'll start doing tasks which we'll be repeating a lot. Aka loading up and managing POT files, generating new rulesets, analyzing new cracks. Therefore it will add a lot to your quality of life to put those tasks in a different Notebook from the previous one-time "setup" tasks. That way when you reload your crack management notebook you won't have to re-run these one-time tasks that no longer matter.

Now I'm not going to do that since this is more of a tutorial, and splitting up a tutorial into multiple Notebooks will make it easeir to follow and learn from. But just keep that in mind as you set up your own cracking sessions.

### Creating a Config File for the Password Cracking Framework

As we move on to actually cracking passwords, we need to create a config file so that SessionMgr of the Framework will have information about what hash files it needs to load, where to store log files, what is the contest point value for different hash types, etc.

What I normally do for this is copy/paste a previoius config I was using and update the hash info. There is an example copy of this that I've invluded as **/challenge_files/CMIYC2024_Test/config.yml**

Here are a couple of areas that you will need to update:

**Location of Main Potfile and Logs**
```
  jtr_config:
    main_pot_file: "./challenge_files/CMIYC2024_Test/jtr_cmiyc2024.pot"
    log_directory: "./challenge_files/CMIYC2024_Test/"
  
  hashcat_config:
    main_pot_file: "./challenge_files/CMIYC2024_Test/hc_cmiyc2024.potfile"
    log_directory: "./challenge_files/CMIYC2024_Test/"
```

**Values to Assign Cracked Hashes**

**Note**: This is taken from the KoreLogic scoreboard
```
  score_info:
    bcrypt: 16777215
    md5crypt: 65535
    nsldaps: 255
    raw-md5: 1
```

In [7]:
from lib_framework.session_mgr import SessionMgr

config_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'config.yml'
    )

sm = SessionMgr(config_file)

Starting to load challenge file: ./challenge_files/CMIYC2024_Test/hashes/test1.hash. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2024_Test/hashes/test_1_passwd.hash. This may take a minute or two
Done loading the challenge file.
